# Test du Modèle Iris avec Service Interne

Ce notebook teste le modèle de classification d'iris en utilisant le service interne du cluster.

In [ ]:
import requests
import json
import numpy as np
import time

In [ ]:
# Configuration du service interne
SERVICE_NAME = "iris-classifier-triton-predictor"
NAMESPACE = "triton-demo"
INTERNAL_URL = f"http://{SERVICE_NAME}.{NAMESPACE}.svc.cluster.local:8080"

print(f"URL interne: {INTERNAL_URL}")
print(f"Service: {SERVICE_NAME}")
print(f"Namespace: {NAMESPACE}")

In [ ]:
# Test de santé du service
try:
    health_response = requests.get(f"{INTERNAL_URL}/v2/health/ready", timeout=10)
    print(f"Health check: {health_response.status_code}")
    if health_response.status_code == 200:
        print("✅ Service en bonne santé")
    else:
        print(f"❌ Problème de santé: {health_response.text}")
except Exception as e:
    print(f"❌ Erreur de connexion: {e}")

In [ ]:
# Test d'inférence simple
sample = np.array([[5.1, 3.5, 1.4, 0.2]], dtype=np.float32)
print(f"Échantillon de test: {sample[0]}")

payload = {
    "inputs": [
        {
            "name": "float_input",
            "shape": sample.shape,
            "datatype": "FP32",
            "data": sample.tolist()
        }
    ]
}

print(f"\nPayload: {json.dumps(payload, indent=2)}")

In [ ]:
# Envoi de la requête d'inférence
try:
    start_time = time.time()
    
    response = requests.post(
        f"{INTERNAL_URL}/v2/models/iris_model/infer",
        headers={"Content-Type": "application/json"},
        json=payload,
        timeout=30
    )
    
    end_time = time.time()
    
    print(f"Status: {response.status_code}")
    print(f"Temps de réponse: {end_time - start_time:.3f} secondes")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Inférence réussie!")
        print(f"   Modèle: {result.get('model_name')}")
        print(f"   Version: {result.get('model_version')}")
        
        # Affichage des résultats
        outputs = result.get('outputs', [])
        for output in outputs:
            if output['name'] == 'label':
                predicted_class = output['data'][0]
                print(f"   Classe prédite: {predicted_class}")
            elif output['name'] == 'probabilities':
                probs = output['data'][0]
                if isinstance(probs, list):
                    print(f"   Probabilités: {[f'{p:.3f}' for p in probs]}")
                else:
                    print(f"   Probabilités: {probs}")
        
        print("\n🎉 Test réussi ! Le modèle fonctionne parfaitement.")
        
    else:
        print(f"❌ Erreur d'inférence: {response.text}")
        
except Exception as e:
    print(f"❌ Erreur: {e}")